In [57]:
import time
start_time = time.time()

In [58]:
import pickle as pkl

with open("../resources/annotated_data.pickle","rb") as pkl_in:
    data = pkl.load(pkl_in)
    text = pkl.load(pkl_in)
    lu = pkl.load(pkl_in)
    pos_tag = pkl.load(pkl_in)
    frame_name = pkl.load(pkl_in)
    frame_element = pkl.load(pkl_in)
    frame_element_lu = pkl.load(pkl_in)
    lang = pkl.load(pkl_in)


In [59]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
def one_hot_encoder(pos_tag_list):
    
    # integer encode
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(pos_tag_list)
    
    # binary encode
    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    
    return onehot_encoded

In [60]:
fname = list(set([token.lower() for k,v in frame_name.items() for token in v]))
felem = list(set([token.lower() for k,v in frame_element.items() for token in v]))

#change all words to lower case
lu_en = [token[0].lower() for token in set(lu["en"])]
lu_pt = [token[0].lower() for token in set(lu["pt"])]
lu_de = [token[0].lower() for token in set(lu["de"])]
fe_lu_en = [token[0].lower() for token in set(frame_element_lu["en"])]
fe_lu_pt = [token[0].lower() for token in set(frame_element_lu["pt"])]
fe_lu_de = [token[0].lower() for token in set(frame_element_lu["de"])]
pos_tag_list = list(set([tag for k,v in pos_tag.items() for tag in v]))

#onehot_encoded = one_hot_encoder(pos_tag_list)
#pos_embed = {tag : onehot_encoded[index] for index, tag in enumerate(pos_tag_list)}

In [61]:
from gensim.models.wrappers import FastText
def fasttext_embedding(path, lang, lu, fe_lu):
    #for lu and frame element lu
    
    lu_embed = {}
    fe_lu_embed = {}
    oov_word = []
    model = FastText.load_fasttext_format(path)
    print('over')
    
    for word in lu:
        try:
            if len(word.split()) > 1 :
                em = []
                for w in word.split():
                    em.append(model[w])
                    
                lu_embed[word] = np.average(em, axis = 0)
                
            else:
                lu_embed[word] = model[word]
        except:
            oov_word.append(word)
    
    for word in fe_lu:
        try:
            if len(word.split()) > 1 :
                em = []
                for w in word.split():
                    em.append(model[w])
                    
                fe_lu_embed[word] = np.average(em, axis = 0)
                
            else:
                fe_lu_embed[word] = model[word]
        except:
            oov_word.append(word)
    
    return lu_embed, fe_lu_embed, oov_word

In [62]:
def frame_embedding(path):
    #frame and frame element embeddings using fasttext
    
    fname_embed = {}
    felem_embed = {}
    oov_word = []
    model = FastText.load_fasttext_format(path)
    print('over')
    
    for word in fname:
        try:
            if len(word.split("_")) > 1 :
                em = []
                for w in word.split("_"):
                    em.append(model[w])

                fname_embed[word] = np.average(em, axis = 0)

            else:
                fname_embed[word] = model[word]
        except:
            oov_word.append(word)

    for word in felem:
        try:
            if len(word.split()) > 1 :
                em = []
                for w in word.split():
                    em.append(model[w])

                felem_embed[word] = np.average(em, axis = 0)

            else:
                felem_embed[word] = model[word]
        except:
            oov_word.append(word)
            
    print(len(oov_word))
    return fname_embed, felem_embed, oov_word

In [9]:
#download the required bin files for the next step from https://fasttext.cc/docs/en/crawl-vectors.html
#provide the path of the downloaded files below
path_en = '../resources/cc.en.300.bin'
path_de = '../resources/cc.de.300.bin'
path_pt = '../resources/cc.pt.300.bin'
fname_embed, felem_embed, oov_frame_en = frame_embedding(path_en)
lu_embed_en, fe_lu_embed_en, oov_word_en = fasttext_embedding(path_en, "en", lu_en, fe_lu_en)
lu_embed_pt, fe_lu_embed_pt, oov_word_pt = fasttext_embedding(path_pt, "pt", lu_pt, fe_lu_pt)
lu_embed_de, fe_lu_embed_de, oov_word_de = fasttext_embedding(path_de, "de", lu_de, fe_lu_de)


/home/Debanjana/anaconda3/envs/gsoc19/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


over
over
over


In [15]:
# Some texts of different lengths in different languages.
english_sentences = text['en']
german_sentences = text['de']
portuguese_sentences = text['pt']

list

##### create sentence embeddings using Google's universal sentence encoder : https://tfhub.dev/google/universal-sentence-encoder-multilingual/1

In [10]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import tf_sentencepiece

In [12]:
# The 8-language multilingual module. There are also en-es, en-de, and en-fr bilingual modules.
module_url = "https://tfhub.dev/google/universal-sentence-encoder-xling-many/1"

# Set up graph.
g = tf.Graph()
with g.as_default():
    text_input = tf.placeholder(dtype=tf.string, shape=[None])
    xling_8_embed = hub.Module(module_url)
    embedded_text = xling_8_embed(text_input)
    init_op = tf.group([tf.global_variables_initializer(), tf.tables_initializer()])
g.finalize()

# Initialize session.
session = tf.Session(graph=g)
session.run(init_op)

# Compute embeddings.
en_result = session.run(embedded_text, feed_dict={text_input: english_sentences})
de_result = session.run(embedded_text, feed_dict={text_input: german_sentences})
pt_result = session.run(embedded_text, feed_dict={text_input: portuguese_sentences})

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0721 22:11:40.355563 140722132289280 tf_logging.py:115] Saver not created because there are no variables in the graph to restore


In [13]:
with open("../resources/frame_embeddings.pickle", "wb") as pkl_fn:
    pkl.dump(fname_embed, pkl_fn)
    pkl.dump(felem_embed, pkl_fn)
    pkl.dump(pos_embed, pkl_fn)
    
with open("../resources/en_embeddings.pickle", "wb") as pkl_en:
    pkl.dump(lu_embed_en, pkl_en)
    pkl.dump(fe_lu_embed_en, pkl_en)
    pkl.dump(en_result, pkl_en)

with open("../resources/de_embeddings.pickle", "wb") as pkl_de:
    pkl.dump(lu_embed_de, pkl_de)
    pkl.dump(fe_lu_embed_de, pkl_de)
    pkl.dump(de_result, pkl_de)
    
with open("../resources/pt_embeddings.pickle", "wb") as pkl_pt:
    pkl.dump(lu_embed_pt, pkl_pt)
    pkl.dump(fe_lu_embed_pt, pkl_pt)
    pkl.dump(pt_result, pkl_pt)

In [14]:
with open('../resources/oov_words_en.txt','w') as f:
    for word in oov_word_en:
        f.write(word + " ")
        
with open('../resources/oov_words_de.txt','w') as f:
    for word in oov_word_de:
        f.write(word + " ")
        
with open('../resources/oov_words_pt.txt','w') as f:
    for word in oov_word_pt:
        f.write(word + " ")

In [15]:
print("--- %s seconds ---" % (time.time() - start_time))

--- 1314.5232484340668 seconds ---
